# Operando con datos en Pandas

Una de las piezas esenciales de NumPy es la capacidad de realizar operaciones rápidas con elementos, tanto con aritmética básica (suma, resta, multiplicación, etc.) como con operaciones más sofisticadas (funciones trigonométricas, funciones exponenciales y logarítmicas, etc.). Pandas hereda gran parte de esta funcionalidad de NumPy, y las ufuncs que presentamos en Computación en arreglos NumPy: funciones universales son clave para esto.


Sin embargo, Pandas incluye un par de giros útiles: para operaciones unarias como negación y funciones trigonométricas, estos ufuncs conservarán las etiquetas de índice y columna en la salida, y para operaciones binarias como suma y multiplicación, Pandas alineará automáticamente los índices al pasar los objetos a el ufunc. Esto significa que mantener el contexto de los datos y combinar datos de diferentes fuentes, ambas tareas potencialmente propensas a errores con matrices NumPy sin formato, se vuelven esencialmente infalibles con Pandas. Veremos además que existen operaciones bien definidas entre operaciones de ``Series`` estructuras unidimensionales  y``DataFrame`` estructuras bidimensionales.

##  Ufuncs: Preservación de índices

Debido a que Pandas está diseñado para funcionar con NumPy, cualquier ufunc de NumPy funcionará en ``Series`` y ``DataFrame`` objetos Pandas. Comencemos definiendo un simple ``Series`` y ``DataFrame`` en que demostrar esto:

In [1]:
import pandas as pd
import numpy as np

In [2]:
rng = np.random.RandomState(42)
ser = pd.Series(rng.randint(0, 10, 4))
ser

0    6
1    3
2    7
3    4
dtype: int64

In [3]:
df = pd.DataFrame(rng.randint(0, 10, (3, 4)),
                  columns=['A', 'B', 'C', 'D'])
df

,A,B,C,D
0,6,9,2,6
1,7,4,3,7
2,7,2,5,4



Si aplicamos un ufunc NumPy en cualquiera de estos objetos, el resultado será otro objeto Pandas con los índices conservados:

In [4]:
np.exp(ser)

0     403.428793
1      20.085537
2    1096.633158
3      54.598150
dtype: float64

O, para un cálculo un poco más complejo:

In [5]:
np.sin(df * np.pi / 4)

,A,B,C,D
0,-1.000000,7.071068e-01,1.000000,-1.000000e+00
1,-0.707107,1.224647e-16,0.707107,-7.071068e-01
2,-0.707107,1.000000e+00,-0.707107,1.224647e-16


Cualquiera de los ufuncs discutidos en Computación en NumPy Arrays: Universal Functions se puede usar de manera similar.

## UFuncs: Alineación de índices

Para operaciones binarias en dos objetos ``Series`` o ``DataFrame``, Pandas alineará los índices en el proceso de realizar la operación. Esto es muy conveniente cuando se trabaja con datos incompletos, como veremos en algunos de los ejemplos que siguen. 

### Alineación de índice en Serie

Como ejemplo, supongamos que estamos combinando dos fuentes de datos diferentes y encontramos solo los tres principales estados de EE. UU. por *área* y los tres principales estados de EE. UU. por *población*:


In [6]:
area = pd.Series({'Alaska': 1723337, 'Texas': 695662,
                  'California': 423967}, name='area')
population = pd.Series({'California': 38332521, 'Texas': 26448193,
                        'New York': 19651127}, name='population')




Veamos qué sucede cuando dividimos estos para calcular la densidad de población:


In [7]:
population / area

Alaska              NaN
California    90.413926
New York            NaN
Texas         38.018740
dtype: float64

La matriz resultante contiene la *unión* de los índices de las dos matrices de entrada, que podrían determinarse utilizando la aritmética de conjuntos estándar de Python en estos índices:

In [8]:
area.index | population.index

/tmp/ipykernel_4545/3572280633.py:1: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  area.index | population.index


Index(['Alaska', 'California', 'New York', 'Texas'], dtype='object')

Cualquier artículo para el cual uno u otro no tiene una entrada se marca con ``NaN``, o "No es un número", que es la forma en que Pandas marca los datos que faltan. Esta coincidencia de índices se implementa de esta manera para cualquiera de las expresiones aritméticas integradas de Python; los valores que faltan se completan con NaN de forma predeterminada:

In [11]:
A = pd.Series([2, 4, 6], index=[0, 1, 2])
B = pd.Series([1, 3, 5], index=[1, 2, 3])
A + B

0    NaN
1    5.0
2    9.0
3    NaN
dtype: float64

Si el uso de valores NaN no es el comportamiento deseado, el valor de relleno se puede modificar mediante los métodos de objetos apropiados en lugar de los operadores.

Llamando ``A.add(B)`` es equivalente a llamar ``A + B``, pero permite la especificación explícita opcional del valor de relleno para cualquier elemento en ``A`` o ``B`` que puede faltar:

In [12]:
A.add(B, fill_value=0)

0    2.0
1    5.0
2    9.0
3    5.0
dtype: float64

### Alineación de índices en DataFrame

Un tipo similar de alineación tiene lugar tanto columnas como para los índices cuando se realizan operaciones en ``DataFrames``:

In [18]:
A = pd.DataFrame(rng.randint(0, 20, (2, 2)),
                 columns=list('AB'))
A

,A,B
0,8,1
1,19,14


In [19]:
B = pd.DataFrame(rng.randint(0, 10, (3, 3)),
                 columns=list('BAC'))
B

,B,A,C
0,6,7,2
1,0,3,1
2,7,3,1


In [20]:
A + B

,A,B,C
0,15.0,7.0,NaN
1,22.0,14.0,NaN
2,NaN,NaN,NaN


Observe que los índices están alineados correctamente independientemente de su orden en los dos objetos, y los índices en el resultado están ordenados. Como fue el caso con ``Series``, podemos usar el método aritmético del objeto asociado y pasar cualquier ``fill_value`` para ser utilizado en lugar de las NaN.

Aquí llenaremos con la media de todos los valores en ``A`` (calculado apilando primero las filas de ``A``):

In [21]:
fill = A.stack().mean()
A.add(B, fill_value=fill)

,A,B,C
0,15.0,7.0,12.5
1,22.0,14.0,11.5
2,13.5,17.5,11.5


La siguiente tabla enumera los operadores de Python y sus métodos de objeto Pandas equivalentes: 

| Python Operator | Pandas Method(s)                      |
|-----------------|---------------------------------------|
| ``+``           | ``add()``                             |
| ``-``           | ``sub()``, ``subtract()``             |
| ``*``           | ``mul()``, ``multiply()``             |
| ``/``           | ``truediv()``, ``div()``, ``divide()``|
| ``//``          | ``floordiv()``                        |
| ``%``           | ``mod()``                             |
| ``**``          | ``pow()``                             |


## Ufuncs: Operaciones entre DataFrame y Series

Al realizar operaciones entre un ``DataFrame`` y una ``Series``, la alineación del índice y la columna se mantiene de manera similar. Operaciones entre un ``DataFrame`` y una ``Series`` son similares a las operaciones entre una matriz NumPy bidimensional y unidimensional. Considere una operación común, donde encontramos la diferencia de una matriz bidimensional y una de sus filas: 

In [25]:
A = rng.randint(10, size=(3, 4))
A

array([[5, 5, 9, 3],
       [5, 1, 9, 1],
       [9, 3, 7, 6]])

In [26]:
A - A[0]

array([[ 0,  0,  0,  0],
       [ 0, -4,  0, -2],
       [ 4, -2, -2,  3]])

De acuerdo con las reglas de broadcasting de NumPy, la resta entre una matriz bidimensional y una de sus filas se aplica por filas.

En Pandas, la convención funciona de manera similar en filas por defecto:

In [29]:
df = pd.DataFrame(A, columns=list('QRST'))
df - df.iloc[0]

,Q,R,S,T
0,0,0,0,0
1,0,-4,0,-2
2,4,-2,-2,3


Si, en cambio, desea operar en forma de columnas, puede usar los métodos de objeto mencionados anteriormente, mientras especifica el ``axis`` palabra clave:

In [30]:
df.subtract(df['R'], axis=0)

,Q,R,S,T
0,0,0,4,-2
1,4,0,8,0
2,6,0,4,3


Tenga en cuenta que estos ``DataFrame``/``Series`` operaciones, como las operaciones discutidas anteriormente, alinearán automáticamente los índices entre los dos elementos:

In [31]:
halfrow = df.iloc[0, ::2]
halfrow

Q    5
S    9
Name: 0, dtype: int64

In [32]:
df - halfrow

,Q,R,S,T
0,0.0,NaN,0.0,NaN
1,0.0,NaN,0.0,NaN
2,4.0,NaN,-2.0,NaN


Esta preservación y alineación de índices y columnas significa que las operaciones en datos en Pandas siempre mantendrán el contexto de datos, lo que evita los tipos de errores tontos que pueden surgir al trabajar con datos heterogéneos y/o desalineados en matrices NumPy sin formato.